# Table of Contents
 <p><div class="lev2 toc-item"><a href="#trial-1" data-toc-modified-id="trial-1-01"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>trial 1</a></div><div class="lev1 toc-item"><a href="#trial-2" data-toc-modified-id="trial-2-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>trial 2</a></div><div class="lev2 toc-item"><a href="#Geometries" data-toc-modified-id="Geometries-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Geometries</a></div>

## trial 1 
### con Rasters 
>como funciona:  
>indicadores ---> modelos  
>modelo --> funcion callback


>Funciones principales a usar:
>    * st_mapAlgebra
    

Como funciona MapAlgebra?

necesitamos una **funcion callback** del tipo:

```sql
DROP FUNCTION callback_intensification(pixel float[], pos integer[], variadic userargs text[]); 
create or replace function callback_intensification(pixel float[], pos integer[], variadic userargs text[]) 
  returns float 
  language plpgsql 
  immutable  -- careful: this function is immutable, yours may not be
as $$
  declare
    pixval float;
    inputx integer;
    inputy integer;
  begin
    pixval := pixel[1][1][1];  -- pixel indices: [raster #][xdistance][ydistance]
    inputx := pos[1][1];       -- pos indices:   [raster #][x = 1, y = 2]
    inputy := pos[1][2];       --                (raster #0 is the output raster)

    return pixval + inputx + inputy;
  end;
$$;
```


y ejecutar esta query

```sql
SELECT ST_MapAlgebra(
		ARRAY[ROW(t1.rast, 3), ROW(t2.rast, 1), ROW(t2.rast, 3), ROW(t1.rast, 2)]::rastbandarg[],
		'sample_callbackfunc(double precision[], int[], text[])'::regprocedure
	) as the_raster_webmercator
```

No funciona :pointup:

# trial 2

## Geometries

In [ ]:
cecsol   cecsol_m_sl1_10km_ll
forest   forest_cover
acy_diff maize_acy_diff
acy_r_h  maize_acy_r_h
acy_r_l  maize_acy_r_l
acy_fut  maize_acy_fut
pop      gpw_v4_population_density_rev10_2020_2pt5_min_10km
wh2      dhs_indicators_wh2
wa2      dhs_indicators_wa2
ha2      dhs_indicators_ha2



```sql
with country as (SELECT the_geom_webmercator FROM all_countries where adm0_a3 ='ETH'), 

wh2 as (select geom as the_geom_webmercator, val as wh2, st_transform(geom,4326) as the_geom from (SELECT (ST_PixelAscentroids(the_raster_webmercator)).* FROM dhs_indicators_wh2 where st_intersects(the_raster_webmercator,(select * from country))) r),
phihox as (select geom as the_geom_webmercator, val as phihox, st_transform(geom,4326) as the_geom from (SELECT (ST_PixelAscentroids(the_raster_webmercator)).* FROM phihox_m_sl1_10km_ll  where st_intersects(the_raster_webmercator,(select * from country))) r),
cecsol as (select geom as the_geom_webmercator, val as cecsol, st_transform(geom,4326) as the_geom from (SELECT (ST_PixelAscentroids(the_raster_webmercator)).* FROM cecsol_m_sl1_10km_ll  where st_intersects(the_raster_webmercator,(select * from country))) r),
forest as (select geom as the_geom_webmercator, val as forest, st_transform(geom,4326) as the_geom from (SELECT (ST_PixelAscentroids(the_raster_webmercator)).* FROM forest_cover  where st_intersects(the_raster_webmercator,(select * from country))) r),
wa2  as (select geom as the_geom_webmercator, val as wa2, st_transform(geom,4326) as the_geom from (SELECT (ST_PixelAscentroids(the_raster_webmercator)).* FROM dhs_indicators_wa2  where st_intersects(the_raster_webmercator,(select * from country))) r),
ha2 as (select geom as the_geom_webmercator, val as ha2, st_transform(geom,4326) as the_geom from (SELECT (ST_PixelAscentroids(the_raster_webmercator)).* FROM dhs_indicators_ha2  where st_intersects(the_raster_webmercator,(select * from country))) r),
pa as (select geom as the_geom_webmercator, val as pa, st_transform(geom,4326) as the_geom from (SELECT (ST_PixelAscentroids(the_raster_webmercator)).* FROM wdpa where st_intersects(the_raster_webmercator,(select * from country))) r),
acy_diff as (select geom as the_geom_webmercator, val as acy_diff, st_transform(geom,4326) as the_geom from (SELECT (ST_PixelAscentroids(the_raster_webmercator)).* FROM maize_acy_diff where st_intersects(the_raster_webmercator,(select * from country))) r)



select model_spect_1_copy.the_geom, model_spect_1_copy.the_geom_webmercator,  wh2.wh2, cecsol, forest, wa2, ha2.ha2, pa.pa,acy_diff.acy_diff from model_spect_1_copy 
inner join wh2 on st_intersects(wh2.the_geom_webmercator, model_spect_1_copy.the_geom_webmercator)
inner join cecsol on st_intersects(cecsol.the_geom_webmercator, model_spect_1_copy.the_geom_webmercator)
inner join forest on st_intersects(forest.the_geom_webmercator, model_spect_1_copy.the_geom_webmercator)
inner join wa2 on st_intersects(wa2.the_geom_webmercator, model_spect_1_copy.the_geom_webmercator)
inner join ha2 on st_intersects(ha2.the_geom_webmercator, model_spect_1_copy.the_geom_webmercator)
inner join pa on st_intersects(pa.the_geom_webmercator, model_spect_1_copy.the_geom_webmercator)
inner join acy_diff on
st_intersects(acy_diff.the_geom_webmercator, model_spect_1_copy.the_geom_webmercator)

```

```json
with country as (SELECT the_geom FROM all_countries where adm0_a3 ='ETH') 

SELECT * FROM model_spect_1 where st_intersects(the_geom, (select * from country))
```

```sql
SELECT the_geom_webmercator, ((acy_diff*1)*(pa*9)*((cecsol*1) + (forest*1) + (ha2*1) + (wa2*1) + (wh2*1))) as value  FROM intensification_reduce
```

```sql
with data as (select * from getModel(1,'[1,1,1,1,1,1,1]') where st_intersects(the_geom, ST_SetSRID (ST_GeomFromGeoJSON('
{"type":"Polygon","coordinates":[[[39.15527343749999,9.178024574578373],[39.495849609375,9.178024574578373],[39.495849609375,9.481572085088517],[39.15527343749999,9.481572085088517],[39.15527343749999,9.178024574578373]]]}'),4326))),

min_max as (select min(value) as min,
           max(value) as max
      from data)


select width_bucket(value, min, max, 20) as bucket,
          min(value), max(value),
          count(*) as freq from data, min_max
 group by bucket
 order by bucket
```

```sql
DROP FUNCTION getModel(table_name TEXT, weights_raw TEXT); 
DROP TYPE weight_model_1;
DROP TYPE weight_input;
CREATE TYPE weight_input AS (column_name text,  weight numeric, operation text);
CREATE TYPE weight_model_1 AS (cartodb_id numeric,  the_geom geometry, the_geom_webmercator geometry, value numeric);

CREATE or REPLACE function getModel(table_name TEXT, weights_raw TEXT) 
RETURNS SETOF weight_model_1 as $$
    DECLARE
    query1 TEXT;
    weights TEXT;
    weights_sum TEXT;
    BEGIN
        query1:='select array_to_string(array_agg(''( ''|| weights::text || '')'' ), '' * '') as weights, sum(weights_sum)::text as weights_sum from (
select array_to_string(array_agg(''( ''|| column_name::text ||'' * ''|| weight::text ||'' ) ''), operation) as weights, sum(weight) as weights_sum, operation from (SELECT (json_populate_recordset(null::weight_input, '' ' || weights_raw || ' '' )).*) as tables where weight is not null group by operation) r';
        EXECUTE query1 into weights, weights_sum;

        RETURN query EXECUTE 'with ranges as (select  average, the_geom, the_geom_webmercator, min(average) over (), max(average) over (), cartodb_id from (SELECT the_geom, the_geom_webmercator, ('|| weights ||')/('|| weights_sum ||') as average, cartodb_id FROM '|| table_name ||') initial) select cartodb_id::numeric,  the_geom, the_geom_webmercator, (((average-min)/(max-min)))::numeric as value from ranges';

    END
$$ language 'plpgsql';
```


Example query:

```sql
select * from getModel('intensification_reduce', '[{"column_name":"wa2", "weight":1, "operation":"+"},{"column_name":"pa", "weight":4, "operation":"*"}]')
```